<a href="https://colab.research.google.com/github/RedDawe/Reinforcement_Chess/blob/master/chessown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu

In [1]:
import tensorflow as tf
print(tf.__version__)
assert tf.executing_eagerly()
import numpy as np

2.0.0


In [0]:
import sys
np.random.seed(0)

In [0]:
  e = 0

  n = 1
  q = 2
  r = 3
  b = 4
  k = 5
  p = 6

  N = -1
  Q = -2
  R = -3
  B = -4
  K = -5
  P = -6

In [0]:
def one_hot(array, n_classes):
  return np.array(tf.keras.backend.one_hot(array, n_classes))

In [0]:
def flip(array):
  return np.flip(array, axis=0)*-1

In [0]:
def init_board():
  board = [[r, k, b, q, n, b, k, r],
          [p, p, p, p, p, p, p, p],
          [e, e, e, e, e, e, e, e],
          [e, e, e, e, e, e, e, e],
          [e, e, e, e, e, e, e, e],
          [e, e, e, e, e, e, e, e],
          [P, P, P, P, P, P, P, P],
          [R, K, B, Q, N, B, K, R]
  ]

  board = np.array(board)

  return board

In [0]:
"""
def weighted_loss(weights):

  def __weighted_loss(logits, labels):
    return tf.keras.backend.mean(tf.keras.backend.square(logits - labels) * weights)
    
  return __weighted_loss
"""

def weighted_loss(logits, labels):
  weights = labels[1, :, :, :]
  labels = labels[0, :, :, :]
  logits = tf.keras.backend.clip(logits, tf.keras.backend.epsilon(), 1-tf.keras.backend.epsilon())

  #return tf.keras.backend.mean(tf.keras.backend.square(logits - labels) * weights)
  return tf.keras.backend.mean(labels * -tf.math.log(logits) * weights)


inputs = tf.keras.Input(shape=(8, 8,  2*p+1), dtype='float32')
#weights = tf.keras.Input(shape=(8, 8, 2*p+1), dtype='float32')

#model = tf.keras.layers.BatchNormalization()(inputs)
model = inputs
#model = tf.keras.backend.one_hot(tf.keras.backend.cast(inputs+p, 'int32'), 2*p+1)
#model = tf.keras.backend.cast(model, 'float32')

for i in range(32):
  cell = tf.keras.layers.Conv2D(filters=2*p+1, kernel_size=[1, 1], activation='elu', padding='same')(model)
  cell = tf.keras.layers.Conv2D(filters=4, kernel_size=[3, 3], activation='elu', padding='same')(cell)
  cell = tf.keras.layers.BatchNormalization()(cell)
  model = tf.keras.layers.Concatenate()(list([model, cell]))

model = tf.keras.layers.Conv2D(filters=2*p+1, kernel_size=[1, 1], activation='linear', padding='same')(model)
model = tf.keras.layers.Reshape([-1, 2*p+1])(model)
model = tf.keras.layers.Softmax(axis=-1)(model)
model = tf.keras.layers.Reshape([8, 8, 2*p+1])(model)

model = tf.keras.Model(inputs=inputs, outputs=model)
#model = tf.keras.Model(inputs=[inputs, weights], outputs=model)


model.compile(optimizer='adam', loss=weighted_loss)
#model.compile(optimizer='adam', loss=weighted_loss(weights))
#model.compile(optimizer='adam', loss='mean_squared_error')
#model.compile(optimizer='adam', loss='categorical_crossentropy')

In [0]:
def make_move(board, side, randomness=1):
  if side:
    board = flip(board)

  #if np.random.randn(1)*0.5+0.5 < randomness:
  if np.random.rand(1) <= randomness:
    #logits =  model.predict([np.expand_dims(board, 0), np.zeros([1, 8, 8])])
    logits =  model.predict([np.expand_dims(one_hot(board+p, 2*p+1), 0), np.zeros([1, 8, 8])])[0, :, :, :]
    piece, to = decide_move(board, logits)
  else:
    piece, to = random_move(board)


  if piece and to:
    """
    if side:
      print(flip(board), piece, to, sep='\n')
    else:
      print(board, piece, to, sep='\n')
    """
    if board[to[0], to[1]] == N or board[to[0], to[1]] == n:
      sys.exit()


    board[to[0], to[1]] = board[piece[0], piece[1]]
    board[piece[0], piece[1]] = e

    weights_arr = np.zeros([8, 8])
    weights_arr[to[0], to[1]] = 1
    weights_arr[piece[0], piece[1]] = 1

    if side:
      return flip(board), np.flip(weights_arr, axis=0)
    else:
      return board, weights_arr
  else:
    return 'game_over', np.zeros([8, 8])


In [0]:
def decide_move(board, logits):
  biggest_diff = 1 #0
  piece = []
  to = []

  for i in range(8):
    for j in range(8):
      #if abs(board[i, j] - logits[i, j]) > biggest_diff:
      #if -(logits[i, j] - board[i, j]) > biggest_diff:
      if board[i, j] > 0:
        if logits[i, j] / board[i, j] < biggest_diff:
          moves = get_possible_moves(board, [i, j])

          for move in moves:
            copy_board = np.copy(board)
            copy_board[move[0], move[1]] = board[i, j]
            copy_board[i, j] = e

            if not is_check(flip(copy_board), N):
              piece = [i, j]
              to = [move[0], move[1]]

  return piece, to

In [0]:
def decide_move(board, logits):
  biggest_diff = 0
  piece = []
  to = []

  for i in range(8):
    for j in range(8):
      if board[i, j] > 0:
        if 1 - logits[i, j, board[i, j]+p] > biggest_diff:
          moves = get_possible_moves(board, [i, j])

          smallest_diff = 0
          for move in moves:
            if logits[move[0], move[1], board[i, j]+p] > smallest_diff:
              copy_board = np.copy(board)
              copy_board[move[0], move[1]] = board[i, j]
              copy_board[i, j] = e

              if not is_check(flip(copy_board), N):
                piece = [i, j]
                to = [move[0], move[1]]

  return piece, to

In [0]:
def random_move(board):
  I = list(range(8))
  J = list(range(8))
  np.random.shuffle(I)
  np.random.shuffle(J)
  piece = []
  to = []

  for i in I:
    for j in J:
      if board[i, j] > 0:
        moves = get_possible_moves(board, [i, j])


        np.random.shuffle(moves)

        for move in moves:
          copy_board = np.copy(board)
          copy_board[move[0], move[1]] = board[i, j]
          copy_board[i, j] = e

          if not is_check(flip(copy_board), N):
            piece = [i, j]
            to = [move[0], move[1]]
            break

        if piece and to:
          break

    if piece and to:
      break

  return piece, to

In [0]:
def is_check(board, piece):
  coordinates = []

  for i in range(8):
    for j in range(8):
      if board[i, j] == piece:
        coordinates = [i, j]

  check = False
  for i in range(8):
    for j in range(8):
      if board[i, j] > 0 and not check:
        moves = get_possible_moves(board, [i, j])
        check = coordinates in moves

  return check

In [0]:
def get_possible_moves(board, coordinates): #put together all moves that 'is_legal', are on the board and aren't taking your own pieces
  legal_moves = []
  for i in range(8):
    for j in range(8):
      if (board[coordinates[0], coordinates[1]] > 0 and board[i, j] <= 0) or (board[coordinates[0], coordinates[1]] < 0 and board[i, j] >= 0):
        if is_legal(board, coordinates, [i, j]):
          legal_moves.append([i, j])

  return legal_moves

In [0]:
def is_legal(board, piece, move): #check if move is in a direction that the piece moves andif squares between start and finish are empty
  if board[piece[0], piece[1]] == n: #KINGS
    return abs(piece[0] - move[0]) <= 1 and abs(piece[1] - move[1]) <= 1

  elif board[piece[0], piece[1]] == q: #QUEEN
    if abs(piece[0] - move[0]) == 0:
      if abs(piece[1] - move[1]) >= 2:
        return np.all(board[min([piece[1], move[1]])+1 : max([piece[1], move[1]])] == e)
      else:
        return True
    elif abs(piece[1] - move[1]) == 0:
      if abs(piece[0] - move[0]) >= 2:
        return np.all(board[min([piece[0], move[0]])+1 : max([piece[0], move[0]])] == e)
      else:
        return True
    elif abs(piece[0] - move[0]) == abs(piece[1] - move[1]):

      distance = abs(piece[0] - move[0])
      vertically = np.arange(distance) * (1 if piece[0] < move[0] else -1)
      horizontally = np.arange(distance) * (1 if piece[1] < move[1] else -1)

      empty = True
      for i in range(1, distance):
        if board[piece[0]+vertically[i], piece[1]+horizontally[i]] != e:
          empty = False

      return empty
    else:
      return False

  elif board[piece[0], piece[1]] == r: #ROOK
    if abs(piece[0] - move[0]) == 0:
      if abs(piece[1] - move[1]) >= 2:
        return np.all(board[min([piece[1], move[1]])+1 : max([piece[1], move[1]])] == e)
      else:
        return True
    elif abs(piece[1] - move[1]) == 0:
      if abs(piece[0] - move[0]) >= 2:
        return np.all(board[min([piece[0], move[0]])+1 : max([piece[0], move[0]])] == e)
      else:
        return True
    else:
      return False

  elif board[piece[0], piece[1]] == b: #BISHOP
    if abs(piece[0] - move[0]) == abs(piece[1] - move[1]):

      distance = abs(piece[0] - move[0])
      vertically = np.arange(distance) * (1 if piece[0] < move[0] else -1)
      horizontally = np.arange(distance) * (1 if piece[1] < move[1] else -1)

      empty = True
      for i in range(1, distance):
        if board[piece[0]+vertically[i], piece[1]+horizontally[i]] != e:
          empty = False

      return empty

    else:
      return False

  elif board[piece[0], piece[1]] == k: #KNIGHT
    return (abs(piece[0] - move[0]) == 2 and abs(piece[0] - move[0]) == 1) or (abs(piece[0] - move[0]) == 1 and abs(piece[0] - move[0]) == 2)

  elif board[piece[0], piece[1]] == p: #PAWN
    return ((move[0] - piece[0]) == 1 and abs(piece[1] - move[1]) == 0 and board[move[0], move[1]] == e) or ((move[0] - piece[0]) == 1 and abs(piece[1] - move[1]) == 1 and board[move[0], move[1]] < 0)

  else:
    print('u fucked up')

In [15]:
kokot = init_board()
kokot[1, 1] = e
print(kokot)
print(is_legal(kokot, [0, 2], [2, 0]))

[[ 3  5  4  2  1  4  5  3]
 [ 6  0  6  6  6  6  6  6]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [-6 -6 -6 -6 -6 -6 -6 -6]
 [-3 -5 -4 -2 -1 -4 -5 -3]]
True


In [0]:
n_steps = 200
games_to_play = 100
max_depth = 100
epochs = 10

In [0]:
checkpoint_path = "training_2/cp.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [0]:
for step in range(0, n_steps):
  print('STEP NUMBER:', step)

  replay_memory = []

  for game in range(games_to_play):
    game_memory = []
    board = init_board()
    #print("NEW GAMEEEEEEEEEEE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

    for depth in range(max_depth):
      move, weights_arr = make_move(np.copy(board), False, step/n_steps)
      if not (type(move) is np.ndarray):
        win = 'black'
        break
      else:
        game_memory.append([board, move, weights_arr])
        board = np.copy(move)

      move, weights_arr = make_move(np.copy(board), True, step/n_steps)
      if not (type(move) is np.ndarray):
        win = 'white'
        break
      else:
        game_memory.append([flip(board), flip(move), np.flip(weights_arr, axis=0)])
        #board = np.copy(flip(move))
        board = np.copy(move)

    else:
      win = 'white' if np.sum(board) > 0 else 'black'

    replay_memory.append([win, game_memory]) 


  x = []
  w = []
  y = []

  for game in replay_memory:
    result, moves = game

    for iterator, data in enumerate(moves):
      board, move, weights_arr = data

      #x.append(one_hot(board, 2*p+1))
      x.append(board)
      #w.append(np.tile(np.expand_dims(weights_arr, -1), [1, 1, 13]))
      w.append(weights_arr)
      if (result == 'white' and iterator%2 == 0) or (result == 'black' and iterator%2 == 1):
        #y.append(one_hot(move+p, 2*p+1))
        y.append(move)
      else:
        #y.append(one_hot(board+p, 2*p+1))
        y.append(board)

  x = np.array(x)
  w = np.array(w)
  y = np.array(y)
  yw = np.stack([y, w], 1)
  
  #model.fit(x=[x, w], y=y, epochs = epochs, callbacks=[cp_callback])
  #model.fit(x=x, y=y, epochs = epochs, callbacks=[cp_callback])
  model.fit(x=x, y=yw, epochs = epochs, callbacks=[cp_callback])

In [0]:
model.load_weights(checkpoint_path)

InvalidArgumentError: ignored

In [0]:
actuall = init_board()
one, two = make_move(actuall, False)
print(one)

[[[0.04045615 0.13503076 0.07136621 0.06308165 0.06515583 0.05310563
   0.06665693 0.13989195 0.05684811 0.10488531 0.05604027 0.05876524
   0.08871595]
  [0.05664025 0.09392781 0.05074177 0.07943127 0.06024208 0.03248367
   0.06649629 0.12422331 0.05417711 0.1353962  0.04079612 0.13407326
   0.07137083]
  [0.08887113 0.06065777 0.08225497 0.09895473 0.05779774 0.04915271
   0.05314228 0.12530081 0.06289372 0.1207284  0.06240023 0.08245656
   0.05538896]
  [0.06388096 0.0943485  0.06695353 0.09049567 0.0457959  0.08131938
   0.04019959 0.16204101 0.05801721 0.07741291 0.04223995 0.11647137
   0.06082403]
  [0.0819439  0.08026277 0.06436513 0.05471571 0.05675073 0.04194325
   0.05021402 0.11261645 0.09046061 0.06109706 0.05561333 0.13834567
   0.11167137]
  [0.04814825 0.08887824 0.05814101 0.08207323 0.05589648 0.05216939
   0.08024685 0.11613324 0.06421359 0.14439712 0.06741749 0.07926934
   0.06301571]
  [0.05348623 0.06594437 0.07090746 0.09713892 0.04563185 0.05286398
   0.04931632

In [0]:
one[6, 4] = e
one[4, 4] = P
one, two = make_move(one, False)
print(one)

[[ 3  5  4  2  1  4  5  0]
 [ 0  6  6  0  6  6  6  3]
 [ 6  0  0  6  0  0  0  6]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0 -6 -6  0  0]
 [ 0  0  0  0  0  0  0  0]
 [-6 -6 -6 -6  0  0 -6 -6]
 [-3 -5 -4 -2 -1 -4 -5 -3]]
